In [1]:
import random
import gymnasium as gym
from pprint import pprint
from catanatron import Color,RandomPlayer
from catanatron.players.weighted_random import WeightedRandomPlayer, DevCardRandomPlayer, DoNothingRandomPlayer,CityRandomPlayer,SettlementRandomPlayer,LongestRoadRandomPlayer
from catanatron_gym.rewards import reward_function
from catanatron_gym.features import create_sample_vector, create_sample
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import json

from catanatron.players.search import VictoryPointPlayer
from catanatron_gym.envs.catanatron_env import from_action_space
import gymnasium as gym
import numpy as np
from sb3_contrib.common.maskable.policies import MaskableActorCriticPolicy
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO

from stable_baselines3 import PPO
from catanatron_gym.features import create_sample_vector, get_feature_ordering,create_sample
from catanatron_gym.envs.catanatron_env import from_action_space
# FEATURES = get_feature_ordering
from pprint import pprint
import random
import pandas as pd
from shaprpy import explain
from catanatron_experimental.machine_learning.players.minimax import AlphaBetaPlayer
import matplotlib.pyplot as plt
# !pip install seaborn
import seaborn as sns
import torch
import torch.nn as nn
from torch import float32
import shap.shap
from torch.autograd import Variable
import os
import re
from rpy2.robjects.vectors import StrVector, ListVector
from rpy2.rinterface import NULL, NA

class sb3Wrapper(nn.Module):
    def __init__(self, model):
        super(sb3Wrapper,self).__init__()
        self.extractor = model.policy.mlp_extractor
        self.policy_net = model.policy.mlp_extractor.policy_net
        self.action_net = model.policy.action_net
        print(self.action_net)

    def forward(self,x):
        x = self.policy_net(x)
        x = self.action_net(x)
        return torch.tensor(torch.argmax(x,keepdim=True),dtype=torch.float64)

def mask_fn(env) -> np.ndarray:
    valid_actions = env.get_valid_actions()
    mask = np.zeros(env.action_space.n, dtype=np.float32)
    mask[valid_actions] = 1

    return np.array([bool(i) for i in mask])

def run_rl_agent(env_config, agent_path= "../logs/Catan_Baseline_rewards_Test/exp_001/latest_model_1000000_steps.zip"):
    # inits for data collection
    try:
        env = gym.make("catanatron_gym:catanatron-v1", env_config)  
        print("made env with config")
    except:
        env = gym.make("catanatron_gym:catanatron-v1")  
        print("made env without config")
    env = ActionMasker(env=env,action_mask_fn=mask_fn)
    observation, info = env.reset()

    model = MaskablePPO.load(path= agent_path,env=env)
    for _ in range(1000):
        action = model.predict(observation=observation,action_masks=mask_fn(env),deterministic=True)

        observation, reward, terminated, truncated, info = env.step(action[0])
        done = terminated or truncated
        if done:
            print(env.game.winning_color())
            observation, info = env.reset()
    # print(info)
    env.close()

def run_switch_agent(env_config, agent_path="./logs/Catan_Switch_Exp/exp_014/best_model.zip"):
    best_path = "../logs/Catan_Switch_Exp/exp_014/best_model.zip"
    latest_path = "../logs/Catan_Switch_Exp/exp_004/latest_model_1000000_steps.zip"
    obs_list = []
    info_list =[]
    action_list =[]
    record_list =[]
    data = []
    features = None
    try:
        env = gym.make("catanatron_gym:catanatron-switch-v1",config = env_config)
        print("made env using config")
    except:
        env = gym.make("catanatron_gym:catanatron-switch-v1")
        print('made env without config')

    observation, info = env.reset()
    model = PPO.load(path=agent_path,env=env)
    # print(env.p0.color)
    # print(model.policy)
    for ep in range(20):
        for step in range(1000):
            action = model.predict(observation=observation,deterministic=True)
            # obs_list.append(observation)
            # action_list.append(action)
            record = create_sample(env.game,env.p0.color)
            features =sorted(record.keys()) # get list of feature names in same order as observation
            data.append([ep,step,observation,action[0]])
            observation, reward, terminated, truncated, info = env.step(action[0])
            done = terminated or truncated
            if done:
                # pprint(vars(env))
                # print(done)
                # print(env.game.winning_color())
                # print(env.game.state.player_state)
                # print(info)
                info_list.append(info)
                observation, info = env.reset()
                break
    # print(info)
    env.close()
    # print(record_list)
    return data, info_list, features

def get_group(features, group):
     # group = {
    #     'a':['BANK_BRICK', 'BANK_DEV_CARDS', 'BANK_ORE', 'BANK_SHEEP', 'BANK_WHEAT', 'BANK_WOOD'],
    #     'b':['EDGE(0, 1)_P0_ROAD', 'EDGE(0, 1)_P1_ROAD', 'EDGE(0, 20)_P0_ROAD', 'EDGE(0, 20)_P1_ROAD', 'EDGE(0, 5)_P0_ROAD', 'EDGE(0, 5)_P1_ROAD', 'EDGE(1, 2)_P0_ROAD', 'EDGE(1, 2)_P1_ROAD', 'EDGE(1, 6)_P0_ROAD', 'EDGE(1, 6)_P1_ROAD', 'EDGE(10, 11)_P0_ROAD', 'EDGE(10, 11)_P1_ROAD', 'EDGE(10, 29)_P0_ROAD', 'EDGE(10, 29)_P1_ROAD', 'EDGE(11, 12)_P0_ROAD', 'EDGE(11, 12)_P1_ROAD', 'EDGE(11, 32)_P0_ROAD', 'EDGE(11, 32)_P1_ROAD', 'EDGE(12, 13)_P0_ROAD', 'EDGE(12, 13)_P1_ROAD', 'EDGE(13, 14)_P0_ROAD', 'EDGE(13, 14)_P1_ROAD', 'EDGE(13, 34)_P0_ROAD', 'EDGE(13, 34)_P1_ROAD', 'EDGE(14, 15)_P0_ROAD', 'EDGE(14, 15)_P1_ROAD', 'EDGE(14, 37)_P0_ROAD', 'EDGE(14, 37)_P1_ROAD', 'EDGE(15, 17)_P0_ROAD', 'EDGE(15, 17)_P1_ROAD', 'EDGE(16, 18)_P0_ROAD', 'EDGE(16, 18)_P1_ROAD', 'EDGE(16, 21)_P0_ROAD', 'EDGE(16, 21)_P1_ROAD', 'EDGE(17, 18)_P0_ROAD', 'EDGE(17, 18)_P1_ROAD', 'EDGE(17, 39)_P0_ROAD', 'EDGE(17, 39)_P1_ROAD', 'EDGE(18, 40)_P0_ROAD', 'EDGE(18, 40)_P1_ROAD', 'EDGE(19, 20)_P0_ROAD', 'EDGE(19, 20)_P1_ROAD', 'EDGE(19, 21)_P0_ROAD', 'EDGE(19, 21)_P1_ROAD', 'EDGE(19, 46)_P0_ROAD', 'EDGE(19, 46)_P1_ROAD', 'EDGE(2, 3)_P0_ROAD', 'EDGE(2, 3)_P1_ROAD', 'EDGE(2, 9)_P0_ROAD', 'EDGE(2, 9)_P1_ROAD', 'EDGE(20, 22)_P0_ROAD', 'EDGE(20, 22)_P1_ROAD', 'EDGE(21, 43)_P0_ROAD', 'EDGE(21, 43)_P1_ROAD', 'EDGE(22, 23)_P0_ROAD', 'EDGE(22, 23)_P1_ROAD', 'EDGE(22, 49)_P0_ROAD', 'EDGE(22, 49)_P1_ROAD', 'EDGE(23, 52)_P0_ROAD', 'EDGE(23, 52)_P1_ROAD', 'EDGE(24, 25)_P0_ROAD', 'EDGE(24, 25)_P1_ROAD', 'EDGE(24, 53)_P0_ROAD', 'EDGE(24, 53)_P1_ROAD', 'EDGE(25, 26)_P0_ROAD', 'EDGE(25, 26)_P1_ROAD', 'EDGE(26, 27)_P0_ROAD', 'EDGE(26, 27)_P1_ROAD', 'EDGE(27, 28)_P0_ROAD', 'EDGE(27, 28)_P1_ROAD', 'EDGE(28, 29)_P0_ROAD', 'EDGE(28, 29)_P1_ROAD', 'EDGE(29, 30)_P0_ROAD', 'EDGE(29, 30)_P1_ROAD', 'EDGE(3, 12)_P0_ROAD', 'EDGE(3, 12)_P1_ROAD', 'EDGE(3, 4)_P0_ROAD', 'EDGE(3, 4)_P1_ROAD', 'EDGE(30, 31)_P0_ROAD', 'EDGE(30, 31)_P1_ROAD', 'EDGE(31, 32)_P0_ROAD', 'EDGE(31, 32)_P1_ROAD', 'EDGE(32, 33)_P0_ROAD', 'EDGE(32, 33)_P1_ROAD', 'EDGE(33, 34)_P0_ROAD', 'EDGE(33, 34)_P1_ROAD', 'EDGE(34, 35)_P0_ROAD', 'EDGE(34, 35)_P1_ROAD', 'EDGE(35, 36)_P0_ROAD', 'EDGE(35, 36)_P1_ROAD', 'EDGE(36, 37)_P0_ROAD', 'EDGE(36, 37)_P1_ROAD', 'EDGE(37, 38)_P0_ROAD', 'EDGE(37, 38)_P1_ROAD', 'EDGE(38, 39)_P0_ROAD', 'EDGE(38, 39)_P1_ROAD', 'EDGE(39, 41)_P0_ROAD', 'EDGE(39, 41)_P1_ROAD', 'EDGE(4, 15)_P0_ROAD', 'EDGE(4, 15)_P1_ROAD', 'EDGE(4, 5)_P0_ROAD', 'EDGE(4, 5)_P1_ROAD', 'EDGE(40, 42)_P0_ROAD', 'EDGE(40, 42)_P1_ROAD', 'EDGE(40, 44)_P0_ROAD', 'EDGE(40, 44)_P1_ROAD', 'EDGE(41, 42)_P0_ROAD', 'EDGE(41, 42)_P1_ROAD', 'EDGE(43, 44)_P0_ROAD', 'EDGE(43, 44)_P1_ROAD', 'EDGE(43, 47)_P0_ROAD', 'EDGE(43, 47)_P1_ROAD', 'EDGE(45, 46)_P0_ROAD', 'EDGE(45, 46)_P1_ROAD', 'EDGE(45, 47)_P0_ROAD', 'EDGE(45, 47)_P1_ROAD', 'EDGE(46, 48)_P0_ROAD', 'EDGE(46, 48)_P1_ROAD', 'EDGE(48, 49)_P0_ROAD', 'EDGE(48, 49)_P1_ROAD', 'EDGE(49, 50)_P0_ROAD', 'EDGE(49, 50)_P1_ROAD', 'EDGE(5, 16)_P0_ROAD', 'EDGE(5, 16)_P1_ROAD', 'EDGE(50, 51)_P0_ROAD', 'EDGE(50, 51)_P1_ROAD', 'EDGE(51, 52)_P0_ROAD', 'EDGE(51, 52)_P1_ROAD', 'EDGE(52, 53)_P0_ROAD', 'EDGE(52, 53)_P1_ROAD', 'EDGE(6, 23)_P0_ROAD', 'EDGE(6, 23)_P1_ROAD', 'EDGE(6, 7)_P0_ROAD', 'EDGE(6, 7)_P1_ROAD', 'EDGE(7, 24)_P0_ROAD', 'EDGE(7, 24)_P1_ROAD', 'EDGE(7, 8)_P0_ROAD', 'EDGE(7, 8)_P1_ROAD', 'EDGE(8, 27)_P0_ROAD', 'EDGE(8, 27)_P1_ROAD', 'EDGE(8, 9)_P0_ROAD', 'EDGE(8, 9)_P1_ROAD', 'EDGE(9, 10)_P0_ROAD', 'EDGE(9, 10)_P1_ROAD'],
    #     'c':['TILE0_HAS_ROBBER', 'TILE0_IS_BRICK', 'TILE0_IS_DESERT', 'TILE0_IS_ORE', 'TILE0_IS_SHEEP', 'TILE0_IS_WHEAT', 'TILE0_IS_WOOD', 'TILE0_PROBA', 'TILE10_HAS_ROBBER', 'TILE10_IS_BRICK', 'TILE10_IS_DESERT', 'TILE10_IS_ORE', 'TILE10_IS_SHEEP', 'TILE10_IS_WHEAT', 'TILE10_IS_WOOD', 'TILE10_PROBA', 'TILE11_HAS_ROBBER', 'TILE11_IS_BRICK', 'TILE11_IS_DESERT', 'TILE11_IS_ORE', 'TILE11_IS_SHEEP', 'TILE11_IS_WHEAT', 'TILE11_IS_WOOD', 'TILE11_PROBA', 'TILE12_HAS_ROBBER', 'TILE12_IS_BRICK', 'TILE12_IS_DESERT', 'TILE12_IS_ORE', 'TILE12_IS_SHEEP', 'TILE12_IS_WHEAT', 'TILE12_IS_WOOD', 'TILE12_PROBA', 'TILE13_HAS_ROBBER', 'TILE13_IS_BRICK', 'TILE13_IS_DESERT', 'TILE13_IS_ORE', 'TILE13_IS_SHEEP', 'TILE13_IS_WHEAT', 'TILE13_IS_WOOD', 'TILE13_PROBA', 'TILE14_HAS_ROBBER', 'TILE14_IS_BRICK', 'TILE14_IS_DESERT', 'TILE14_IS_ORE', 'TILE14_IS_SHEEP', 'TILE14_IS_WHEAT', 'TILE14_IS_WOOD', 'TILE14_PROBA', 'TILE15_HAS_ROBBER', 'TILE15_IS_BRICK', 'TILE15_IS_DESERT', 'TILE15_IS_ORE', 'TILE15_IS_SHEEP', 'TILE15_IS_WHEAT', 'TILE15_IS_WOOD', 'TILE15_PROBA', 'TILE16_HAS_ROBBER', 'TILE16_IS_BRICK', 'TILE16_IS_DESERT', 'TILE16_IS_ORE', 'TILE16_IS_SHEEP', 'TILE16_IS_WHEAT', 'TILE16_IS_WOOD', 'TILE16_PROBA', 'TILE17_HAS_ROBBER', 'TILE17_IS_BRICK', 'TILE17_IS_DESERT', 'TILE17_IS_ORE', 'TILE17_IS_SHEEP', 'TILE17_IS_WHEAT', 'TILE17_IS_WOOD', 'TILE17_PROBA', 'TILE18_HAS_ROBBER', 'TILE18_IS_BRICK', 'TILE18_IS_DESERT', 'TILE18_IS_ORE', 'TILE18_IS_SHEEP', 'TILE18_IS_WHEAT', 'TILE18_IS_WOOD', 'TILE18_PROBA', 'TILE1_HAS_ROBBER', 'TILE1_IS_BRICK', 'TILE1_IS_DESERT', 'TILE1_IS_ORE', 'TILE1_IS_SHEEP', 'TILE1_IS_WHEAT', 'TILE1_IS_WOOD', 'TILE1_PROBA', 'TILE2_HAS_ROBBER', 'TILE2_IS_BRICK', 'TILE2_IS_DESERT', 'TILE2_IS_ORE', 'TILE2_IS_SHEEP', 'TILE2_IS_WHEAT', 'TILE2_IS_WOOD', 'TILE2_PROBA', 'TILE3_HAS_ROBBER', 'TILE3_IS_BRICK', 'TILE3_IS_DESERT', 'TILE3_IS_ORE', 'TILE3_IS_SHEEP', 'TILE3_IS_WHEAT', 'TILE3_IS_WOOD', 'TILE3_PROBA', 'TILE4_HAS_ROBBER', 'TILE4_IS_BRICK', 'TILE4_IS_DESERT', 'TILE4_IS_ORE', 'TILE4_IS_SHEEP', 'TILE4_IS_WHEAT', 'TILE4_IS_WOOD', 'TILE4_PROBA', 'TILE5_HAS_ROBBER', 'TILE5_IS_BRICK', 'TILE5_IS_DESERT', 'TILE5_IS_ORE', 'TILE5_IS_SHEEP', 'TILE5_IS_WHEAT', 'TILE5_IS_WOOD', 'TILE5_PROBA', 'TILE6_HAS_ROBBER', 'TILE6_IS_BRICK', 'TILE6_IS_DESERT', 'TILE6_IS_ORE', 'TILE6_IS_SHEEP', 'TILE6_IS_WHEAT', 'TILE6_IS_WOOD', 'TILE6_PROBA', 'TILE7_HAS_ROBBER', 'TILE7_IS_BRICK', 'TILE7_IS_DESERT', 'TILE7_IS_ORE', 'TILE7_IS_SHEEP', 'TILE7_IS_WHEAT', 'TILE7_IS_WOOD', 'TILE7_PROBA', 'TILE8_HAS_ROBBER', 'TILE8_IS_BRICK', 'TILE8_IS_DESERT', 'TILE8_IS_ORE', 'TILE8_IS_SHEEP', 'TILE8_IS_WHEAT', 'TILE8_IS_WOOD', 'TILE8_PROBA', 'TILE9_HAS_ROBBER', 'TILE9_IS_BRICK', 'TILE9_IS_DESERT', 'TILE9_IS_ORE', 'TILE9_IS_SHEEP', 'TILE9_IS_WHEAT', 'TILE9_IS_WOOD', 'TILE9_PROBA'],
    #     'd':['NODE0_P0_CITY', 'NODE0_P0_SETTLEMENT', 'NODE0_P1_CITY', 'NODE0_P1_SETTLEMENT', 'NODE10_P0_CITY', 'NODE10_P0_SETTLEMENT', 'NODE10_P1_CITY', 'NODE10_P1_SETTLEMENT', 'NODE11_P0_CITY', 'NODE11_P0_SETTLEMENT', 'NODE11_P1_CITY', 'NODE11_P1_SETTLEMENT', 'NODE12_P0_CITY', 'NODE12_P0_SETTLEMENT', 'NODE12_P1_CITY', 'NODE12_P1_SETTLEMENT', 'NODE13_P0_CITY', 'NODE13_P0_SETTLEMENT', 'NODE13_P1_CITY', 'NODE13_P1_SETTLEMENT', 'NODE14_P0_CITY', 'NODE14_P0_SETTLEMENT', 'NODE14_P1_CITY', 'NODE14_P1_SETTLEMENT', 'NODE15_P0_CITY', 'NODE15_P0_SETTLEMENT', 'NODE15_P1_CITY', 'NODE15_P1_SETTLEMENT', 'NODE16_P0_CITY', 'NODE16_P0_SETTLEMENT', 'NODE16_P1_CITY', 'NODE16_P1_SETTLEMENT', 'NODE17_P0_CITY', 'NODE17_P0_SETTLEMENT', 'NODE17_P1_CITY', 'NODE17_P1_SETTLEMENT', 'NODE18_P0_CITY', 'NODE18_P0_SETTLEMENT', 'NODE18_P1_CITY', 'NODE18_P1_SETTLEMENT', 'NODE19_P0_CITY', 'NODE19_P0_SETTLEMENT', 'NODE19_P1_CITY', 'NODE19_P1_SETTLEMENT', 'NODE1_P0_CITY', 'NODE1_P0_SETTLEMENT', 'NODE1_P1_CITY', 'NODE1_P1_SETTLEMENT', 'NODE20_P0_CITY', 'NODE20_P0_SETTLEMENT', 'NODE20_P1_CITY', 'NODE20_P1_SETTLEMENT', 'NODE21_P0_CITY', 'NODE21_P0_SETTLEMENT', 'NODE21_P1_CITY', 'NODE21_P1_SETTLEMENT', 'NODE22_P0_CITY', 'NODE22_P0_SETTLEMENT', 'NODE22_P1_CITY', 'NODE22_P1_SETTLEMENT', 'NODE23_P0_CITY', 'NODE23_P0_SETTLEMENT', 'NODE23_P1_CITY', 'NODE23_P1_SETTLEMENT', 'NODE24_P0_CITY', 'NODE24_P0_SETTLEMENT', 'NODE24_P1_CITY', 'NODE24_P1_SETTLEMENT', 'NODE25_P0_CITY', 'NODE25_P0_SETTLEMENT', 'NODE25_P1_CITY', 'NODE25_P1_SETTLEMENT', 'NODE26_P0_CITY', 'NODE26_P0_SETTLEMENT', 'NODE26_P1_CITY', 'NODE26_P1_SETTLEMENT', 'NODE27_P0_CITY', 'NODE27_P0_SETTLEMENT', 'NODE27_P1_CITY', 'NODE27_P1_SETTLEMENT', 'NODE28_P0_CITY', 'NODE28_P0_SETTLEMENT', 'NODE28_P1_CITY', 'NODE28_P1_SETTLEMENT', 'NODE29_P0_CITY', 'NODE29_P0_SETTLEMENT', 'NODE29_P1_CITY', 'NODE29_P1_SETTLEMENT', 'NODE2_P0_CITY', 'NODE2_P0_SETTLEMENT', 'NODE2_P1_CITY', 'NODE2_P1_SETTLEMENT', 'NODE30_P0_CITY', 'NODE30_P0_SETTLEMENT', 'NODE30_P1_CITY', 'NODE30_P1_SETTLEMENT', 'NODE31_P0_CITY', 'NODE31_P0_SETTLEMENT', 'NODE31_P1_CITY', 'NODE31_P1_SETTLEMENT', 'NODE32_P0_CITY', 'NODE32_P0_SETTLEMENT', 'NODE32_P1_CITY', 'NODE32_P1_SETTLEMENT', 'NODE33_P0_CITY', 'NODE33_P0_SETTLEMENT', 'NODE33_P1_CITY', 'NODE33_P1_SETTLEMENT', 'NODE34_P0_CITY', 'NODE34_P0_SETTLEMENT', 'NODE34_P1_CITY', 'NODE34_P1_SETTLEMENT', 'NODE35_P0_CITY', 'NODE35_P0_SETTLEMENT', 'NODE35_P1_CITY', 'NODE35_P1_SETTLEMENT', 'NODE36_P0_CITY', 'NODE36_P0_SETTLEMENT', 'NODE36_P1_CITY', 'NODE36_P1_SETTLEMENT', 'NODE37_P0_CITY', 'NODE37_P0_SETTLEMENT', 'NODE37_P1_CITY', 'NODE37_P1_SETTLEMENT', 'NODE38_P0_CITY', 'NODE38_P0_SETTLEMENT', 'NODE38_P1_CITY', 'NODE38_P1_SETTLEMENT', 'NODE39_P0_CITY', 'NODE39_P0_SETTLEMENT', 'NODE39_P1_CITY', 'NODE39_P1_SETTLEMENT', 'NODE3_P0_CITY', 'NODE3_P0_SETTLEMENT', 'NODE3_P1_CITY', 'NODE3_P1_SETTLEMENT', 'NODE40_P0_CITY', 'NODE40_P0_SETTLEMENT', 'NODE40_P1_CITY', 'NODE40_P1_SETTLEMENT', 'NODE41_P0_CITY', 'NODE41_P0_SETTLEMENT', 'NODE41_P1_CITY', 'NODE41_P1_SETTLEMENT', 'NODE42_P0_CITY', 'NODE42_P0_SETTLEMENT', 'NODE42_P1_CITY', 'NODE42_P1_SETTLEMENT', 'NODE43_P0_CITY', 'NODE43_P0_SETTLEMENT', 'NODE43_P1_CITY', 'NODE43_P1_SETTLEMENT', 'NODE44_P0_CITY', 'NODE44_P0_SETTLEMENT', 'NODE44_P1_CITY', 'NODE44_P1_SETTLEMENT', 'NODE45_P0_CITY', 'NODE45_P0_SETTLEMENT', 'NODE45_P1_CITY', 'NODE45_P1_SETTLEMENT', 'NODE46_P0_CITY', 'NODE46_P0_SETTLEMENT', 'NODE46_P1_CITY', 'NODE46_P1_SETTLEMENT', 'NODE47_P0_CITY', 'NODE47_P0_SETTLEMENT', 'NODE47_P1_CITY', 'NODE47_P1_SETTLEMENT', 'NODE48_P0_CITY', 'NODE48_P0_SETTLEMENT', 'NODE48_P1_CITY', 'NODE48_P1_SETTLEMENT', 'NODE49_P0_CITY', 'NODE49_P0_SETTLEMENT', 'NODE49_P1_CITY', 'NODE49_P1_SETTLEMENT', 'NODE4_P0_CITY', 'NODE4_P0_SETTLEMENT', 'NODE4_P1_CITY', 'NODE4_P1_SETTLEMENT', 'NODE50_P0_CITY', 'NODE50_P0_SETTLEMENT', 'NODE50_P1_CITY', 'NODE50_P1_SETTLEMENT', 'NODE51_P0_CITY', 'NODE51_P0_SETTLEMENT', 'NODE51_P1_CITY', 'NODE51_P1_SETTLEMENT', 'NODE52_P0_CITY', 'NODE52_P0_SETTLEMENT', 'NODE52_P1_CITY', 'NODE52_P1_SETTLEMENT', 'NODE53_P0_CITY', 'NODE53_P0_SETTLEMENT', 'NODE53_P1_CITY', 'NODE53_P1_SETTLEMENT', 'NODE5_P0_CITY', 'NODE5_P0_SETTLEMENT', 'NODE5_P1_CITY', 'NODE5_P1_SETTLEMENT', 'NODE6_P0_CITY', 'NODE6_P0_SETTLEMENT', 'NODE6_P1_CITY', 'NODE6_P1_SETTLEMENT', 'NODE7_P0_CITY', 'NODE7_P0_SETTLEMENT', 'NODE7_P1_CITY', 'NODE7_P1_SETTLEMENT', 'NODE8_P0_CITY', 'NODE8_P0_SETTLEMENT', 'NODE8_P1_CITY', 'NODE8_P1_SETTLEMENT', 'NODE9_P0_CITY', 'NODE9_P0_SETTLEMENT', 'NODE9_P1_CITY', 'NODE9_P1_SETTLEMENT'],
    #     'e':['PORT0_IS_BRICK', 'PORT0_IS_ORE', 'PORT0_IS_SHEEP', 'PORT0_IS_THREE_TO_ONE', 'PORT0_IS_WHEAT', 'PORT0_IS_WOOD', 'PORT1_IS_BRICK', 'PORT1_IS_ORE', 'PORT1_IS_SHEEP', 'PORT1_IS_THREE_TO_ONE', 'PORT1_IS_WHEAT', 'PORT1_IS_WOOD', 'PORT2_IS_BRICK', 'PORT2_IS_ORE', 'PORT2_IS_SHEEP', 'PORT2_IS_THREE_TO_ONE', 'PORT2_IS_WHEAT', 'PORT2_IS_WOOD', 'PORT3_IS_BRICK', 'PORT3_IS_ORE', 'PORT3_IS_SHEEP', 'PORT3_IS_THREE_TO_ONE', 'PORT3_IS_WHEAT', 'PORT3_IS_WOOD', 'PORT4_IS_BRICK', 'PORT4_IS_ORE', 'PORT4_IS_SHEEP', 'PORT4_IS_THREE_TO_ONE', 'PORT4_IS_WHEAT', 'PORT4_IS_WOOD', 'PORT5_IS_BRICK', 'PORT5_IS_ORE', 'PORT5_IS_SHEEP', 'PORT5_IS_THREE_TO_ONE', 'PORT5_IS_WHEAT', 'PORT5_IS_WOOD', 'PORT6_IS_BRICK', 'PORT6_IS_ORE', 'PORT6_IS_SHEEP', 'PORT6_IS_THREE_TO_ONE', 'PORT6_IS_WHEAT', 'PORT6_IS_WOOD', 'PORT7_IS_BRICK', 'PORT7_IS_ORE', 'PORT7_IS_SHEEP', 'PORT7_IS_THREE_TO_ONE', 'PORT7_IS_WHEAT', 'PORT7_IS_WOOD', 'PORT8_IS_BRICK', 'PORT8_IS_ORE', 'PORT8_IS_SHEEP', 'PORT8_IS_THREE_TO_ONE', 'PORT8_IS_WHEAT', 'PORT8_IS_WOOD'],
    #     'f':['IS_DISCARDING', 'IS_MOVING_ROBBER'],
    #     'g':['P0_HAS_ROLLED', 'P1_HAS_ROLLED'],
    #     'h':['P0_HAS_PLAYED_DEVELOPMENT_CARD_IN_TURN'],
    #     'i':['P0_ACTUAL_VPS', 'P0_PUBLIC_VPS'],
    #     'j':['P0_BRICK_IN_HAND', 'P0_KNIGHT_IN_HAND', 'P0_MONOPOLY_IN_HAND', 'P0_NUM_DEVS_IN_HAND', 'P0_NUM_RESOURCES_IN_HAND', 'P0_ORE_IN_HAND', 'P0_ROAD_BUILDING_IN_HAND', 'P0_SHEEP_IN_HAND', 'P0_VICTORY_POINT_IN_HAND', 'P0_WHEAT_IN_HAND', 'P0_WOOD_IN_HAND', 'P0_YEAR_OF_PLENTY_IN_HAND'],
    #     'k':['P1_NUM_DEVS_IN_HAND', 'P1_NUM_RESOURCES_IN_HAND'],
    #     'l':['P0_HAS_ARMY'],
    #     'm':['P1_HAS_ARMY'],
    #     'n':['P0_HAS_ROAD'],
    #     'o':['P1_HAS_ROAD'],
    #     'p':['P0_CITIES_LEFT', 'P0_ROADS_LEFT', 'P0_SETTLEMENTS_LEFT'],
    #     'q':['P1_CITIES_LEFT', 'P1_ROADS_LEFT', 'P1_SETTLEMENTS_LEFT'],
    #     'r':['P0_KNIGHT_PLAYED', 'P0_MONOPOLY_PLAYED', 'P0_ROAD_BUILDING_PLAYED', 'P0_YEAR_OF_PLENTY_PLAYED'],
    #     's':['P1_KNIGHT_PLAYED', 'P1_MONOPOLY_PLAYED', 'P1_ROAD_BUILDING_PLAYED', 'P1_YEAR_OF_PLENTY_PLAYED'],
    #     't':['P1_PUBLIC_VPS'],
    #     'u':['P0_LONGEST_ROAD_LENGTH'],
    #     'v':['P1_LONGEST_ROAD_LENGTH'],
    # }
    bank_features = re.compile(".*BANK")
    edge_features = re.compile(".*EDGE")
    tile_features = re.compile(".*TILE")
    node_features = re.compile(".*NODE")
    port_features = re.compile(".*PORT")
    rolled_robber_features = re.compile("(.*IS_DISCARDING)|(.*IS_MOVING_ROBBER)")
    rolled_feature = re.compile(".*HAS_ROLLED")
    p0_played_dev_in_turn = re.compile(r'P0_HAS_PLAYED_DEVELOPMENT_CARD_IN_TURN')
    p0_vps = re.compile(".*P0_ACTUAL_VPS|.*P0_PUBLIC_VPS")
    p0_hand = re.compile(r'P0_[A-Za-z0-9_]+_IN_HAND')
    p1_hand = re.compile(r'P1_[A-Za-z0-9_]+_IN_HAND')
    p0_has_army = re.compile(r'P0_HAS_ARMY')
    p0_has_road = re.compile(r'P0_HAS_ROAD')
    p1_has_army = re.compile(r'P1_HAS_ARMY')
    p1_has_road = re.compile(r'P1_HAS_ROAD')
    p0_pieces= re.compile(r'P0_[A-Za-z0-9_]+_LEFT')
    p1_pieces= re.compile(r'P1_[A-Za-z0-9_]+_LEFT')
    p0_devs_played = re.compile(r'P0_[A-Za-z0-9_]+_PLAYED$')
    p1_devs_played = re.compile(r'P1_[A-Za-z0-9_]+_PLAYED')
    p1_public_vps = re.compile(r'P1_PUBLIC_VPS')
    p0_longest_road_length = re.compile(r'P0_LONGEST_ROAD_LENGTH')
    p1_longest_road_length = re.compile(r'P1_LONGEST_ROAD_LENGTH')


    bank_features =list(filter(bank_features.match, features))
    edge_features =list(filter(edge_features.match, features))
    tile_features =list(filter(tile_features.match, features))
    node_features=list(filter(node_features.match, features))
    port_features=list(filter(port_features.match, features))
    rolled_robber_features=list(filter(rolled_robber_features.match, features))
    rolled_feature=list(filter(rolled_feature.match, features))
    p0_played_dev_in_turn =list(filter(p0_played_dev_in_turn.match, features))
    p0_vps = list(filter(p0_vps.match, features))
    p0_hand = list(filter(p0_hand.match, features))
    p1_hand =list(filter(p1_hand.match, features))
    p0_has_army =list(filter(p0_has_army.match, features))
    p1_has_army=list(filter(p1_has_army.match, features))
    p0_has_road=list(filter(p0_has_road.match, features))
    p1_has_road=list(filter(p1_has_road.match, features))
    p0_pieces =list(filter(p0_pieces.match, features))
    p1_pieces=list(filter(p1_pieces.match, features))
    p0_devs_played =list(filter(p0_devs_played.match, features))
    p1_devs_played =list(filter(p1_devs_played.match, features))
    p1_public_vps=list(filter(p1_public_vps.match, features))
    p0_longest_road_length=list(filter(p0_longest_road_length.match, features))
    p1_longest_road_length=list(filter(p1_longest_road_length.match, features))


    # as a group
    if group == "group_1":
        group_1 = {
            "bank": bank_features,
            "edge_features": edge_features,
            "tile_features": tile_features,
            "node_features": node_features,
            "port_features":port_features,
            "rolled_robber_features": rolled_robber_features, # ['IS_DISCARDING', 'IS_MOVING_ROBBER'],
            "rolled_feature": rolled_feature, # ['P0_HAS_ROLLED', 'P1_HAS_ROLLED'],
            "p0_played_dev_in_turn": p0_played_dev_in_turn, # ['P0_HAS_PLAYED_DEVELOPMENT_CARD_IN_TURN'],
            "p0_vps": p0_vps, # ['P0_ACTUAL_VPS', 'P0_PUBLIC_VPS'],
            "p1_public_vps": p1_public_vps, # ['P0_PUBLIC_VPS'],
            "p0_hand": p0_hand, # ['P0_BRICK_IN_HAND', 'P0_KNIGHT_IN_HAND', 'P0_MONOPOLY_IN_HAND', 'P0_NUM_DEVS_IN_HAND', 'P0_NUM_RESOURCES_IN_HAND', 'P0_ORE_IN_HAND', 'P0_ROAD_BUILDING_IN_HAND', 'P0_SHEEP_IN_HAND', 'P0_VICTORY_POINT_IN_HAND', 'P0_WHEAT_IN_HAND', 'P0_WOOD_IN_HAND', 'P0_YEAR_OF_PLENTY_IN_HAND'],
            "p1_hand": p1_hand, # ['P1_NUM_DEVS_IN_HAND', 'P1_NUM_RESOURCES_IN_HAND'],
            "p0_has_army": p0_has_army, #['P0_HAS_ARMY'],
            "p0_has_longest_road": p0_has_road,# ['P0_HAS_ROAD'],
            "p0_longest_road_length": p0_longest_road_length,# ['P0_LONGEST_ROAD_LENGTH'],
            "p1_has_army":p1_has_army, # ['P1_HAS_ARMY'],
            "p1_has_longest_road": p1_has_road,# ['P1_HAS_ROAD'],
            "p1_longest_road_length":p1_longest_road_length,# ['P1_LONGEST_ROAD_LENGTH'],
            "p0_pieces": p0_pieces, # ['P0_CITIES_LEFT', 'P0_ROADS_LEFT', 'P0_SETTLEMENTS_LEFT'],
            "p1_pieces":p1_pieces, #['P1_CITIES_LEFT', 'P1_ROADS_LEFT', 'P1_SETTLEMENTS_LEFT'],
            "p0_devs_played":p0_devs_played, # ['P0_KNIGHT_PLAYED', 'P0_MONOPOLY_PLAYED', 'P0_ROAD_BUILDING_PLAYED', 'P0_YEAR_OF_PLENTY_PLAYED'],
            "p1_devs_played": p1_devs_played, # ['P1_KNIGHT_PLAYED', 'P1_MONOPOLY_PLAYED', 'P1_ROAD_BUILDING_PLAYED', 'P1_YEAR_OF_PLENTY_PLAYED'],
        }
        return group_1
    elif group == "group_2":
        group_2 ={
            "board_features": bank_features + tile_features + port_features, 
            "graph_features": node_features + edge_features,
            "turn_features": rolled_feature + rolled_robber_features+p0_played_dev_in_turn,
            "p0_vps": p0_vps + p0_has_army + p0_has_road, #TODO Get the VP_in hand
            "p1_vps": p1_public_vps + p1_has_army + p1_has_road,
            "p0_hand_and_pieces": p0_hand + p0_pieces + p0_longest_road_length,
            "p1_hand_and_pieces": p1_hand + p1_pieces +p1_longest_road_length,
            "p0_dev_cards_played": p0_devs_played,
            "p1_dev_cards_played": p1_devs_played,
        }
        return group_2
    elif group == "group_3":
        group_3 ={
            'board_features': bank_features + tile_features + port_features, 
            'graph_features': node_features + edge_features,
            'turn_features': rolled_feature + rolled_robber_features+p0_played_dev_in_turn,
            'p0_vps': p0_vps + p0_has_army + p0_has_road+ p0_hand + p0_pieces + p0_longest_road_length + p0_devs_played,
            'p1_vps': p1_public_vps + p1_has_army + p1_has_road + p1_hand + p1_pieces +p1_longest_road_length + p1_devs_played
        }
        return group_3
    else:
        return None

def prep_data(data:pd.DataFrame,features):
    data = data.copy()
    df = data[['obs','action']]
    data_list =data.obs.tolist()
    dfx = pd.DataFrame(data_list,columns=features ,index = data.index)
    print(df)
    #find len of final ep
    #TODO find length of winning episode
    # len(df[df.ep ==df.ep.max()])

    dfx_train, dfx_test, dfy_train, dfy_test = train_test_split(dfx[features],df['action'],shuffle=False, test_size=len(data[data.ep ==data.ep.max()]))
    #suffle the training data
    dfx_train, dfy_train = shuffle(dfx_train,dfy_train, random_state=42)
    return dfx_train, dfx_test, dfy_train, dfy_test

def shapley(env_config= None, agent_path = "./logs/Catan_Switch_Exp/exp_014/best_model.zip"):
    data, info_list, catan_features = run_switch_agent(env_config=env_config, agent_path=agent_path)
    features = ["ep","step",'obs', 'action']
    df = pd.DataFrame(data=data, columns=features)

    #format data
    # dfx_train, dfx_test, dfy_train, dfy_test = prep_data(df,catan_features)

    #load_model
    
    model = PPO.load(path =agent_path, device='cuda')
    print(type(model))
    # model = sb3Wrapper(model)
    print(type(model))
    print(model)
    # print(model.action_net)
    # print(dfy_train.mean())
    # model.predict()[0]
    # explainer = shap.shap.DeepExplainer(model,dfx_train,framework='pytorch')
    state_log = np.array(df['obs'].values.tolist())
    tensor_data = torch.from_numpy(state_log).to('cuda')
    print(tensor_data)
    print(tensor_data.shape)
    # f = lambda x: model.forward(Variable(torch.from_numpy(x),requires_grad=False).to(float32).cuda()).detach().cpu().numpy()
    
    f = lambda x: model.predict(x,deterministic=True)[0]
    print(f(state_log[0]))
    explainer = shap.shap.KernelExplainer(f, state_log)
    #TODO get shapvals for single episode and look into sampling and saving models
    shap_vals= explainer.shap_values(state_log)
    # state_log = np.array(df['obs'].values.tolist())


    # tensor_data = torch.FloatTensor(state_log).to('cuda')
    # print(tensor_data[0])
    # print(model.forward(tensor_data[0]).dtype)
    # print(df['action'][0])
    # explainer = shap.DeepExplainer(model,tensor_data)
    # shap_values = explainer.shap_values(data,check_additivity=True)

    return shap_vals

def groupshap(env_config= None, approach:str ="independence", grouping =None, agent_path = "./logs/Catan_Switch_Exp/exp_014/best_model.zip"):
    data, info_list, catan_features = run_switch_agent(env_config=env_config, agent_path=agent_path)
    features = ["ep","step",'obs', 'action']
    df = pd.DataFrame(data=data, columns=features)
    group = get_group(catan_features, grouping)
    # print(group)

    #format data
    dfx_train, dfx_test, dfy_train, dfy_test = prep_data(df,catan_features)
    # print(dfx_train.dtypes)
    # print(dfx_train.columns.values.tolist())
    # print(len(catan_features))
    r_group = NULL if group is None else ListVector({key: StrVector(value) for key, value in group.items()})
    # print(r_group)

    # group = {
    #     'a':['BANK_BRICK', 'BANK_DEV_CARDS', 'BANK_ORE', 'BANK_SHEEP', 'BANK_WHEAT', 'BANK_WOOD'],
    #     'b':['EDGE(0, 1)_P0_ROAD', 'EDGE(0, 1)_P1_ROAD', 'EDGE(0, 20)_P0_ROAD', 'EDGE(0, 20)_P1_ROAD', 'EDGE(0, 5)_P0_ROAD', 'EDGE(0, 5)_P1_ROAD', 'EDGE(1, 2)_P0_ROAD', 'EDGE(1, 2)_P1_ROAD', 'EDGE(1, 6)_P0_ROAD', 'EDGE(1, 6)_P1_ROAD', 'EDGE(10, 11)_P0_ROAD', 'EDGE(10, 11)_P1_ROAD', 'EDGE(10, 29)_P0_ROAD', 'EDGE(10, 29)_P1_ROAD', 'EDGE(11, 12)_P0_ROAD', 'EDGE(11, 12)_P1_ROAD', 'EDGE(11, 32)_P0_ROAD', 'EDGE(11, 32)_P1_ROAD', 'EDGE(12, 13)_P0_ROAD', 'EDGE(12, 13)_P1_ROAD', 'EDGE(13, 14)_P0_ROAD', 'EDGE(13, 14)_P1_ROAD', 'EDGE(13, 34)_P0_ROAD', 'EDGE(13, 34)_P1_ROAD', 'EDGE(14, 15)_P0_ROAD', 'EDGE(14, 15)_P1_ROAD', 'EDGE(14, 37)_P0_ROAD', 'EDGE(14, 37)_P1_ROAD', 'EDGE(15, 17)_P0_ROAD', 'EDGE(15, 17)_P1_ROAD', 'EDGE(16, 18)_P0_ROAD', 'EDGE(16, 18)_P1_ROAD', 'EDGE(16, 21)_P0_ROAD', 'EDGE(16, 21)_P1_ROAD', 'EDGE(17, 18)_P0_ROAD', 'EDGE(17, 18)_P1_ROAD', 'EDGE(17, 39)_P0_ROAD', 'EDGE(17, 39)_P1_ROAD', 'EDGE(18, 40)_P0_ROAD', 'EDGE(18, 40)_P1_ROAD', 'EDGE(19, 20)_P0_ROAD', 'EDGE(19, 20)_P1_ROAD', 'EDGE(19, 21)_P0_ROAD', 'EDGE(19, 21)_P1_ROAD', 'EDGE(19, 46)_P0_ROAD', 'EDGE(19, 46)_P1_ROAD', 'EDGE(2, 3)_P0_ROAD', 'EDGE(2, 3)_P1_ROAD', 'EDGE(2, 9)_P0_ROAD', 'EDGE(2, 9)_P1_ROAD', 'EDGE(20, 22)_P0_ROAD', 'EDGE(20, 22)_P1_ROAD', 'EDGE(21, 43)_P0_ROAD', 'EDGE(21, 43)_P1_ROAD', 'EDGE(22, 23)_P0_ROAD', 'EDGE(22, 23)_P1_ROAD', 'EDGE(22, 49)_P0_ROAD', 'EDGE(22, 49)_P1_ROAD', 'EDGE(23, 52)_P0_ROAD', 'EDGE(23, 52)_P1_ROAD', 'EDGE(24, 25)_P0_ROAD', 'EDGE(24, 25)_P1_ROAD', 'EDGE(24, 53)_P0_ROAD', 'EDGE(24, 53)_P1_ROAD', 'EDGE(25, 26)_P0_ROAD', 'EDGE(25, 26)_P1_ROAD', 'EDGE(26, 27)_P0_ROAD', 'EDGE(26, 27)_P1_ROAD', 'EDGE(27, 28)_P0_ROAD', 'EDGE(27, 28)_P1_ROAD', 'EDGE(28, 29)_P0_ROAD', 'EDGE(28, 29)_P1_ROAD', 'EDGE(29, 30)_P0_ROAD', 'EDGE(29, 30)_P1_ROAD', 'EDGE(3, 12)_P0_ROAD', 'EDGE(3, 12)_P1_ROAD', 'EDGE(3, 4)_P0_ROAD', 'EDGE(3, 4)_P1_ROAD', 'EDGE(30, 31)_P0_ROAD', 'EDGE(30, 31)_P1_ROAD', 'EDGE(31, 32)_P0_ROAD', 'EDGE(31, 32)_P1_ROAD', 'EDGE(32, 33)_P0_ROAD', 'EDGE(32, 33)_P1_ROAD', 'EDGE(33, 34)_P0_ROAD', 'EDGE(33, 34)_P1_ROAD', 'EDGE(34, 35)_P0_ROAD', 'EDGE(34, 35)_P1_ROAD', 'EDGE(35, 36)_P0_ROAD', 'EDGE(35, 36)_P1_ROAD', 'EDGE(36, 37)_P0_ROAD', 'EDGE(36, 37)_P1_ROAD', 'EDGE(37, 38)_P0_ROAD', 'EDGE(37, 38)_P1_ROAD', 'EDGE(38, 39)_P0_ROAD', 'EDGE(38, 39)_P1_ROAD', 'EDGE(39, 41)_P0_ROAD', 'EDGE(39, 41)_P1_ROAD', 'EDGE(4, 15)_P0_ROAD', 'EDGE(4, 15)_P1_ROAD', 'EDGE(4, 5)_P0_ROAD', 'EDGE(4, 5)_P1_ROAD', 'EDGE(40, 42)_P0_ROAD', 'EDGE(40, 42)_P1_ROAD', 'EDGE(40, 44)_P0_ROAD', 'EDGE(40, 44)_P1_ROAD', 'EDGE(41, 42)_P0_ROAD', 'EDGE(41, 42)_P1_ROAD', 'EDGE(43, 44)_P0_ROAD', 'EDGE(43, 44)_P1_ROAD', 'EDGE(43, 47)_P0_ROAD', 'EDGE(43, 47)_P1_ROAD', 'EDGE(45, 46)_P0_ROAD', 'EDGE(45, 46)_P1_ROAD', 'EDGE(45, 47)_P0_ROAD', 'EDGE(45, 47)_P1_ROAD', 'EDGE(46, 48)_P0_ROAD', 'EDGE(46, 48)_P1_ROAD', 'EDGE(48, 49)_P0_ROAD', 'EDGE(48, 49)_P1_ROAD', 'EDGE(49, 50)_P0_ROAD', 'EDGE(49, 50)_P1_ROAD', 'EDGE(5, 16)_P0_ROAD', 'EDGE(5, 16)_P1_ROAD', 'EDGE(50, 51)_P0_ROAD', 'EDGE(50, 51)_P1_ROAD', 'EDGE(51, 52)_P0_ROAD', 'EDGE(51, 52)_P1_ROAD', 'EDGE(52, 53)_P0_ROAD', 'EDGE(52, 53)_P1_ROAD', 'EDGE(6, 23)_P0_ROAD', 'EDGE(6, 23)_P1_ROAD', 'EDGE(6, 7)_P0_ROAD', 'EDGE(6, 7)_P1_ROAD', 'EDGE(7, 24)_P0_ROAD', 'EDGE(7, 24)_P1_ROAD', 'EDGE(7, 8)_P0_ROAD', 'EDGE(7, 8)_P1_ROAD', 'EDGE(8, 27)_P0_ROAD', 'EDGE(8, 27)_P1_ROAD', 'EDGE(8, 9)_P0_ROAD', 'EDGE(8, 9)_P1_ROAD', 'EDGE(9, 10)_P0_ROAD', 'EDGE(9, 10)_P1_ROAD'],
    #     'c':['TILE0_HAS_ROBBER', 'TILE0_IS_BRICK', 'TILE0_IS_DESERT', 'TILE0_IS_ORE', 'TILE0_IS_SHEEP', 'TILE0_IS_WHEAT', 'TILE0_IS_WOOD', 'TILE0_PROBA', 'TILE10_HAS_ROBBER', 'TILE10_IS_BRICK', 'TILE10_IS_DESERT', 'TILE10_IS_ORE', 'TILE10_IS_SHEEP', 'TILE10_IS_WHEAT', 'TILE10_IS_WOOD', 'TILE10_PROBA', 'TILE11_HAS_ROBBER', 'TILE11_IS_BRICK', 'TILE11_IS_DESERT', 'TILE11_IS_ORE', 'TILE11_IS_SHEEP', 'TILE11_IS_WHEAT', 'TILE11_IS_WOOD', 'TILE11_PROBA', 'TILE12_HAS_ROBBER', 'TILE12_IS_BRICK', 'TILE12_IS_DESERT', 'TILE12_IS_ORE', 'TILE12_IS_SHEEP', 'TILE12_IS_WHEAT', 'TILE12_IS_WOOD', 'TILE12_PROBA', 'TILE13_HAS_ROBBER', 'TILE13_IS_BRICK', 'TILE13_IS_DESERT', 'TILE13_IS_ORE', 'TILE13_IS_SHEEP', 'TILE13_IS_WHEAT', 'TILE13_IS_WOOD', 'TILE13_PROBA', 'TILE14_HAS_ROBBER', 'TILE14_IS_BRICK', 'TILE14_IS_DESERT', 'TILE14_IS_ORE', 'TILE14_IS_SHEEP', 'TILE14_IS_WHEAT', 'TILE14_IS_WOOD', 'TILE14_PROBA', 'TILE15_HAS_ROBBER', 'TILE15_IS_BRICK', 'TILE15_IS_DESERT', 'TILE15_IS_ORE', 'TILE15_IS_SHEEP', 'TILE15_IS_WHEAT', 'TILE15_IS_WOOD', 'TILE15_PROBA', 'TILE16_HAS_ROBBER', 'TILE16_IS_BRICK', 'TILE16_IS_DESERT', 'TILE16_IS_ORE', 'TILE16_IS_SHEEP', 'TILE16_IS_WHEAT', 'TILE16_IS_WOOD', 'TILE16_PROBA', 'TILE17_HAS_ROBBER', 'TILE17_IS_BRICK', 'TILE17_IS_DESERT', 'TILE17_IS_ORE', 'TILE17_IS_SHEEP', 'TILE17_IS_WHEAT', 'TILE17_IS_WOOD', 'TILE17_PROBA', 'TILE18_HAS_ROBBER', 'TILE18_IS_BRICK', 'TILE18_IS_DESERT', 'TILE18_IS_ORE', 'TILE18_IS_SHEEP', 'TILE18_IS_WHEAT', 'TILE18_IS_WOOD', 'TILE18_PROBA', 'TILE1_HAS_ROBBER', 'TILE1_IS_BRICK', 'TILE1_IS_DESERT', 'TILE1_IS_ORE', 'TILE1_IS_SHEEP', 'TILE1_IS_WHEAT', 'TILE1_IS_WOOD', 'TILE1_PROBA', 'TILE2_HAS_ROBBER', 'TILE2_IS_BRICK', 'TILE2_IS_DESERT', 'TILE2_IS_ORE', 'TILE2_IS_SHEEP', 'TILE2_IS_WHEAT', 'TILE2_IS_WOOD', 'TILE2_PROBA', 'TILE3_HAS_ROBBER', 'TILE3_IS_BRICK', 'TILE3_IS_DESERT', 'TILE3_IS_ORE', 'TILE3_IS_SHEEP', 'TILE3_IS_WHEAT', 'TILE3_IS_WOOD', 'TILE3_PROBA', 'TILE4_HAS_ROBBER', 'TILE4_IS_BRICK', 'TILE4_IS_DESERT', 'TILE4_IS_ORE', 'TILE4_IS_SHEEP', 'TILE4_IS_WHEAT', 'TILE4_IS_WOOD', 'TILE4_PROBA', 'TILE5_HAS_ROBBER', 'TILE5_IS_BRICK', 'TILE5_IS_DESERT', 'TILE5_IS_ORE', 'TILE5_IS_SHEEP', 'TILE5_IS_WHEAT', 'TILE5_IS_WOOD', 'TILE5_PROBA', 'TILE6_HAS_ROBBER', 'TILE6_IS_BRICK', 'TILE6_IS_DESERT', 'TILE6_IS_ORE', 'TILE6_IS_SHEEP', 'TILE6_IS_WHEAT', 'TILE6_IS_WOOD', 'TILE6_PROBA', 'TILE7_HAS_ROBBER', 'TILE7_IS_BRICK', 'TILE7_IS_DESERT', 'TILE7_IS_ORE', 'TILE7_IS_SHEEP', 'TILE7_IS_WHEAT', 'TILE7_IS_WOOD', 'TILE7_PROBA', 'TILE8_HAS_ROBBER', 'TILE8_IS_BRICK', 'TILE8_IS_DESERT', 'TILE8_IS_ORE', 'TILE8_IS_SHEEP', 'TILE8_IS_WHEAT', 'TILE8_IS_WOOD', 'TILE8_PROBA', 'TILE9_HAS_ROBBER', 'TILE9_IS_BRICK', 'TILE9_IS_DESERT', 'TILE9_IS_ORE', 'TILE9_IS_SHEEP', 'TILE9_IS_WHEAT', 'TILE9_IS_WOOD', 'TILE9_PROBA'],
    #     'd':['NODE0_P0_CITY', 'NODE0_P0_SETTLEMENT', 'NODE0_P1_CITY', 'NODE0_P1_SETTLEMENT', 'NODE10_P0_CITY', 'NODE10_P0_SETTLEMENT', 'NODE10_P1_CITY', 'NODE10_P1_SETTLEMENT', 'NODE11_P0_CITY', 'NODE11_P0_SETTLEMENT', 'NODE11_P1_CITY', 'NODE11_P1_SETTLEMENT', 'NODE12_P0_CITY', 'NODE12_P0_SETTLEMENT', 'NODE12_P1_CITY', 'NODE12_P1_SETTLEMENT', 'NODE13_P0_CITY', 'NODE13_P0_SETTLEMENT', 'NODE13_P1_CITY', 'NODE13_P1_SETTLEMENT', 'NODE14_P0_CITY', 'NODE14_P0_SETTLEMENT', 'NODE14_P1_CITY', 'NODE14_P1_SETTLEMENT', 'NODE15_P0_CITY', 'NODE15_P0_SETTLEMENT', 'NODE15_P1_CITY', 'NODE15_P1_SETTLEMENT', 'NODE16_P0_CITY', 'NODE16_P0_SETTLEMENT', 'NODE16_P1_CITY', 'NODE16_P1_SETTLEMENT', 'NODE17_P0_CITY', 'NODE17_P0_SETTLEMENT', 'NODE17_P1_CITY', 'NODE17_P1_SETTLEMENT', 'NODE18_P0_CITY', 'NODE18_P0_SETTLEMENT', 'NODE18_P1_CITY', 'NODE18_P1_SETTLEMENT', 'NODE19_P0_CITY', 'NODE19_P0_SETTLEMENT', 'NODE19_P1_CITY', 'NODE19_P1_SETTLEMENT', 'NODE1_P0_CITY', 'NODE1_P0_SETTLEMENT', 'NODE1_P1_CITY', 'NODE1_P1_SETTLEMENT', 'NODE20_P0_CITY', 'NODE20_P0_SETTLEMENT', 'NODE20_P1_CITY', 'NODE20_P1_SETTLEMENT', 'NODE21_P0_CITY', 'NODE21_P0_SETTLEMENT', 'NODE21_P1_CITY', 'NODE21_P1_SETTLEMENT', 'NODE22_P0_CITY', 'NODE22_P0_SETTLEMENT', 'NODE22_P1_CITY', 'NODE22_P1_SETTLEMENT', 'NODE23_P0_CITY', 'NODE23_P0_SETTLEMENT', 'NODE23_P1_CITY', 'NODE23_P1_SETTLEMENT', 'NODE24_P0_CITY', 'NODE24_P0_SETTLEMENT', 'NODE24_P1_CITY', 'NODE24_P1_SETTLEMENT', 'NODE25_P0_CITY', 'NODE25_P0_SETTLEMENT', 'NODE25_P1_CITY', 'NODE25_P1_SETTLEMENT', 'NODE26_P0_CITY', 'NODE26_P0_SETTLEMENT', 'NODE26_P1_CITY', 'NODE26_P1_SETTLEMENT', 'NODE27_P0_CITY', 'NODE27_P0_SETTLEMENT', 'NODE27_P1_CITY', 'NODE27_P1_SETTLEMENT', 'NODE28_P0_CITY', 'NODE28_P0_SETTLEMENT', 'NODE28_P1_CITY', 'NODE28_P1_SETTLEMENT', 'NODE29_P0_CITY', 'NODE29_P0_SETTLEMENT', 'NODE29_P1_CITY', 'NODE29_P1_SETTLEMENT', 'NODE2_P0_CITY', 'NODE2_P0_SETTLEMENT', 'NODE2_P1_CITY', 'NODE2_P1_SETTLEMENT', 'NODE30_P0_CITY', 'NODE30_P0_SETTLEMENT', 'NODE30_P1_CITY', 'NODE30_P1_SETTLEMENT', 'NODE31_P0_CITY', 'NODE31_P0_SETTLEMENT', 'NODE31_P1_CITY', 'NODE31_P1_SETTLEMENT', 'NODE32_P0_CITY', 'NODE32_P0_SETTLEMENT', 'NODE32_P1_CITY', 'NODE32_P1_SETTLEMENT', 'NODE33_P0_CITY', 'NODE33_P0_SETTLEMENT', 'NODE33_P1_CITY', 'NODE33_P1_SETTLEMENT', 'NODE34_P0_CITY', 'NODE34_P0_SETTLEMENT', 'NODE34_P1_CITY', 'NODE34_P1_SETTLEMENT', 'NODE35_P0_CITY', 'NODE35_P0_SETTLEMENT', 'NODE35_P1_CITY', 'NODE35_P1_SETTLEMENT', 'NODE36_P0_CITY', 'NODE36_P0_SETTLEMENT', 'NODE36_P1_CITY', 'NODE36_P1_SETTLEMENT', 'NODE37_P0_CITY', 'NODE37_P0_SETTLEMENT', 'NODE37_P1_CITY', 'NODE37_P1_SETTLEMENT', 'NODE38_P0_CITY', 'NODE38_P0_SETTLEMENT', 'NODE38_P1_CITY', 'NODE38_P1_SETTLEMENT', 'NODE39_P0_CITY', 'NODE39_P0_SETTLEMENT', 'NODE39_P1_CITY', 'NODE39_P1_SETTLEMENT', 'NODE3_P0_CITY', 'NODE3_P0_SETTLEMENT', 'NODE3_P1_CITY', 'NODE3_P1_SETTLEMENT', 'NODE40_P0_CITY', 'NODE40_P0_SETTLEMENT', 'NODE40_P1_CITY', 'NODE40_P1_SETTLEMENT', 'NODE41_P0_CITY', 'NODE41_P0_SETTLEMENT', 'NODE41_P1_CITY', 'NODE41_P1_SETTLEMENT', 'NODE42_P0_CITY', 'NODE42_P0_SETTLEMENT', 'NODE42_P1_CITY', 'NODE42_P1_SETTLEMENT', 'NODE43_P0_CITY', 'NODE43_P0_SETTLEMENT', 'NODE43_P1_CITY', 'NODE43_P1_SETTLEMENT', 'NODE44_P0_CITY', 'NODE44_P0_SETTLEMENT', 'NODE44_P1_CITY', 'NODE44_P1_SETTLEMENT', 'NODE45_P0_CITY', 'NODE45_P0_SETTLEMENT', 'NODE45_P1_CITY', 'NODE45_P1_SETTLEMENT', 'NODE46_P0_CITY', 'NODE46_P0_SETTLEMENT', 'NODE46_P1_CITY', 'NODE46_P1_SETTLEMENT', 'NODE47_P0_CITY', 'NODE47_P0_SETTLEMENT', 'NODE47_P1_CITY', 'NODE47_P1_SETTLEMENT', 'NODE48_P0_CITY', 'NODE48_P0_SETTLEMENT', 'NODE48_P1_CITY', 'NODE48_P1_SETTLEMENT', 'NODE49_P0_CITY', 'NODE49_P0_SETTLEMENT', 'NODE49_P1_CITY', 'NODE49_P1_SETTLEMENT', 'NODE4_P0_CITY', 'NODE4_P0_SETTLEMENT', 'NODE4_P1_CITY', 'NODE4_P1_SETTLEMENT', 'NODE50_P0_CITY', 'NODE50_P0_SETTLEMENT', 'NODE50_P1_CITY', 'NODE50_P1_SETTLEMENT', 'NODE51_P0_CITY', 'NODE51_P0_SETTLEMENT', 'NODE51_P1_CITY', 'NODE51_P1_SETTLEMENT', 'NODE52_P0_CITY', 'NODE52_P0_SETTLEMENT', 'NODE52_P1_CITY', 'NODE52_P1_SETTLEMENT', 'NODE53_P0_CITY', 'NODE53_P0_SETTLEMENT', 'NODE53_P1_CITY', 'NODE53_P1_SETTLEMENT', 'NODE5_P0_CITY', 'NODE5_P0_SETTLEMENT', 'NODE5_P1_CITY', 'NODE5_P1_SETTLEMENT', 'NODE6_P0_CITY', 'NODE6_P0_SETTLEMENT', 'NODE6_P1_CITY', 'NODE6_P1_SETTLEMENT', 'NODE7_P0_CITY', 'NODE7_P0_SETTLEMENT', 'NODE7_P1_CITY', 'NODE7_P1_SETTLEMENT', 'NODE8_P0_CITY', 'NODE8_P0_SETTLEMENT', 'NODE8_P1_CITY', 'NODE8_P1_SETTLEMENT', 'NODE9_P0_CITY', 'NODE9_P0_SETTLEMENT', 'NODE9_P1_CITY', 'NODE9_P1_SETTLEMENT'],
    #     'e':['PORT0_IS_BRICK', 'PORT0_IS_ORE', 'PORT0_IS_SHEEP', 'PORT0_IS_THREE_TO_ONE', 'PORT0_IS_WHEAT', 'PORT0_IS_WOOD', 'PORT1_IS_BRICK', 'PORT1_IS_ORE', 'PORT1_IS_SHEEP', 'PORT1_IS_THREE_TO_ONE', 'PORT1_IS_WHEAT', 'PORT1_IS_WOOD', 'PORT2_IS_BRICK', 'PORT2_IS_ORE', 'PORT2_IS_SHEEP', 'PORT2_IS_THREE_TO_ONE', 'PORT2_IS_WHEAT', 'PORT2_IS_WOOD', 'PORT3_IS_BRICK', 'PORT3_IS_ORE', 'PORT3_IS_SHEEP', 'PORT3_IS_THREE_TO_ONE', 'PORT3_IS_WHEAT', 'PORT3_IS_WOOD', 'PORT4_IS_BRICK', 'PORT4_IS_ORE', 'PORT4_IS_SHEEP', 'PORT4_IS_THREE_TO_ONE', 'PORT4_IS_WHEAT', 'PORT4_IS_WOOD', 'PORT5_IS_BRICK', 'PORT5_IS_ORE', 'PORT5_IS_SHEEP', 'PORT5_IS_THREE_TO_ONE', 'PORT5_IS_WHEAT', 'PORT5_IS_WOOD', 'PORT6_IS_BRICK', 'PORT6_IS_ORE', 'PORT6_IS_SHEEP', 'PORT6_IS_THREE_TO_ONE', 'PORT6_IS_WHEAT', 'PORT6_IS_WOOD', 'PORT7_IS_BRICK', 'PORT7_IS_ORE', 'PORT7_IS_SHEEP', 'PORT7_IS_THREE_TO_ONE', 'PORT7_IS_WHEAT', 'PORT7_IS_WOOD', 'PORT8_IS_BRICK', 'PORT8_IS_ORE', 'PORT8_IS_SHEEP', 'PORT8_IS_THREE_TO_ONE', 'PORT8_IS_WHEAT', 'PORT8_IS_WOOD'],
    #     'f':['IS_DISCARDING', 'IS_MOVING_ROBBER'],
    #     'g':['P0_HAS_ROLLED', 'P1_HAS_ROLLED'],
    #     'h':['P0_HAS_PLAYED_DEVELOPMENT_CARD_IN_TURN'],
    #     'i':['P0_ACTUAL_VPS', 'P0_PUBLIC_VPS'],
    #     'j':['P0_BRICK_IN_HAND', 'P0_KNIGHT_IN_HAND', 'P0_MONOPOLY_IN_HAND', 'P0_NUM_DEVS_IN_HAND', 'P0_NUM_RESOURCES_IN_HAND', 'P0_ORE_IN_HAND', 'P0_ROAD_BUILDING_IN_HAND', 'P0_SHEEP_IN_HAND', 'P0_VICTORY_POINT_IN_HAND', 'P0_WHEAT_IN_HAND', 'P0_WOOD_IN_HAND', 'P0_YEAR_OF_PLENTY_IN_HAND'],
    #     'k':['P1_NUM_DEVS_IN_HAND', 'P1_NUM_RESOURCES_IN_HAND'],
    #     'l':['P0_HAS_ARMY'],
    #     'm':['P1_HAS_ARMY'],
    #     'n':['P0_HAS_ROAD'],
    #     'o':['P1_HAS_ROAD'],
    #     'p':['P0_CITIES_LEFT', 'P0_ROADS_LEFT', 'P0_SETTLEMENTS_LEFT'],
    #     'q':['P1_CITIES_LEFT', 'P1_ROADS_LEFT', 'P1_SETTLEMENTS_LEFT'],
    #     'r':['P0_KNIGHT_PLAYED', 'P0_MONOPOLY_PLAYED', 'P0_ROAD_BUILDING_PLAYED', 'P0_YEAR_OF_PLENTY_PLAYED'],
    #     's':['P1_KNIGHT_PLAYED', 'P1_MONOPOLY_PLAYED', 'P1_ROAD_BUILDING_PLAYED', 'P1_YEAR_OF_PLENTY_PLAYED'],
    #     't':['P1_PUBLIC_VPS'],
    #     'u':['P0_LONGEST_ROAD_LENGTH'],
    #     'v':['P1_LONGEST_ROAD_LENGTH'],
    # }
    #load_model
    model = PPO.load(path =agent_path, device='cuda')
    print(dfy_train.mean())
    # model.predict()[0]

    df_shapley, pred_explain, internal, timing ,MSEv= explain(
        model = model,
        x_train = dfx_train,
        x_explain = dfx_test,
        approach = approach,
        # n_combinations=2**614,
        n_samples=200,
        # n_batches=10,
        predict_model= lambda m,x: m.predict(x,deterministic =True)[0],
        prediction_zero = dfy_train.mean().item(),
        group=group,
        verbose=1,
    )
    return df_shapley, pred_explain, internal, timing, MSEv


def test_shap():
    from sklearn.ensemble import RandomForestRegressor
    from shaprpy import explain
    from shaprpy.datasets import load_california_housing

    dfx_train, dfx_test, dfy_train, dfy_test = load_california_housing()

    ## Fit model
    model = RandomForestRegressor()
    model.fit(dfx_train, dfy_train.values.flatten())

    ## Shapr
    df_shapley, pred_explain, internal, timing ,MSEv= explain(
        model = model,
        x_train = dfx_train,
        x_explain = dfx_test,
        approach = 'empirical',
        prediction_zero = dfy_train.mean().item(),
        verbose= 1
    )
    print(df_shapley)


    

In [2]:
config={"enemies": [AlphaBetaPlayer(Color.RED,depth=1)]}
#normal shap
# shap_vals = shapley(config)
# np.save('./shapley_data/shapley_vals.npy', shap_vals)
#group_shapley for group 3 (the smallest group) 
df_shapley, pred_explain, internal, timing, MSEv = groupshap(env_config=config, approach= 'empirical', grouping = "group_3")


running switch env
{'enemies': [AlphaBetaPlayer:RED(depth=1,value_fn=base_fn,prunning=False)]}
made env using config
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.game to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.game` for environment variables or `env.get_wrapper_attr('game')` that will search the reminding wrappers.
  logger.warn(
/home/alex/anaconda3/envs/catanatron/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.p0 to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.p0` for environment variables or `env.get_wrapper_attr('p0')` that will search the reminding wrappers.
  logger.warn(


                                                    obs action
0     [19.0, 25.0, 19.0, 19.0, 19.0, 19.0, 0.0, 0.0,...      4
1     [19.0, 25.0, 19.0, 19.0, 19.0, 19.0, 0.0, 0.0,...      4
2     [19.0, 25.0, 18.0, 19.0, 18.0, 18.0, 0.0, 0.0,...      4
3     [19.0, 25.0, 18.0, 18.0, 18.0, 16.0, 0.0, 0.0,...      4
4     [19.0, 25.0, 18.0, 18.0, 18.0, 16.0, 0.0, 0.0,...      4
...                                                 ...    ...
1564  [17.0, 23.0, 17.0, 17.0, 17.0, 18.0, 0.0, 0.0,...      1
1565  [17.0, 23.0, 17.0, 17.0, 17.0, 18.0, 0.0, 0.0,...      1
1566  [18.0, 23.0, 17.0, 17.0, 17.0, 19.0, 0.0, 0.0,...      4
1567  [16.0, 23.0, 17.0, 17.0, 17.0, 19.0, 0.0, 0.0,...      4
1568  [16.0, 23.0, 17.0, 17.0, 17.0, 17.0, 0.0, 0.0,...      1

[1569 rows x 2 columns]
2.6568694463431304


/home/alex/Masters/catanatron/shapr/python/shaprpy/explain.py:255: UserWarning: No pre-built get_model_specs for model of type <class 'stable_baselines3.ppo.ppo.PPO'>, disabling checks.
  warnings.warn(f'No pre-built get_model_specs for model of type {type(model)}, disabling checks.')
R[write to console]: Note: You passed a model to explain() which is not natively supported, and did not supply a 'get_model_specs' function to explain().
Consistency checks between model and data is therefore disabled.


R[write to console]: Setting parameter 'n_batches' to 2 as a fair trade-off between memory consumption and computation time.
Reducing 'n_batches' typically reduces the computation time at the cost of increased memory consumption.




In [68]:
# class json_encoder(json.JSONEncoder):
#     def default(self,obj):
#         if isinstance(obj, np.integer):
#             return int(obj)
#         elif isinstance(obj, np.floating):
#             return float(obj)
#         elif isinstance(obj, np.ndarray):
#             return obj.tolist()
#         elif isinstance(obj, pd.DataFrame):
#             return obj.where(pd.notnull(obj), None).to_dict(orient='list')
#         else:
#             return super(json_encoder, self).default(obj)

df_shapley.to_csv(path_or_buf='./shapley_data/groupshap_group_3test.csv')
print("pred_explain is numpy with shape:",pred_explain.shape)
np.save(file ='./shapley_data/pred_explain_group_3_test.npy',arr=pred_explain)
# print("internal is dict\n",internal)
# with open("./shapley_data/internal_group_3.json", "w") as file:
#     json.dump(internal,file,indent=4)   
print("timing is dict:\n",timing)
with open("./shapley_data/timing_group_3.json", "w") as file:
    json.dump(timing,fp=file,indent=4)
print("msev is dict:\n",MSEv)
with open("./shapley_data/MSEv_group_3_test.json", "w") as file:
    json.dump({'MSEv':MSEv["MSEv"].to_dict(orient='records'),"MSEv_explicand":MSEv["MSEv_explicand"].to_dict(orient='records')},fp =file, indent=4)

pred_explain is numpy with shape: (106,)
timing is dict:
 {'init_time': '2024-09-10 09:42:48', 'total_time_secs': 664.2968359999999, 'timing_secs': {'setup': 1.458739, 'test_prediction': 0.113681, 'compute_vS': 662.719216, 'shapley_computation': 0.0052}}
msev is dict:
 {'MSEv':        MSEv   MSEv_sd
1  1.405588  0.068192, 'MSEv_explicand':       id      MSEv
1      1  0.714104
2      2  1.366293
3      3  1.364806
4      4  1.366302
5      5  1.112899
..   ...       ...
102  102  2.807689
103  103  1.754769
104  104  1.699362
105  105  1.704878
106  106  2.996771

[106 rows x 2 columns], 'MSEv_combination':     id_combination  (features, board_features1)  (features, board_features2)  \
1                2                          1.0                          2.0   
2                3                          NaN                          NaN   
3                4                          NaN                          NaN   
4                5                          NaN                  